In [1]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000


In [4]:
with open("sarcasm.json", 'r') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])

In [5]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [6]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [7]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d_1  (None, 16)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_2 (Dense)             (None, 24)                408       
                                                                 
 dense_3 (Dense)             (None, 1)                 25        
                                                                 
Total params: 160433 (626.69 KB)
Trainable params: 160433 (626.69 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [10]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 1s - loss: 0.6535 - accuracy: 0.5993 - val_loss: 0.5508 - val_accuracy: 0.7480 - 785ms/epoch - 1ms/step
Epoch 2/30
625/625 - 1s - loss: 0.4100 - accuracy: 0.8389 - val_loss: 0.3746 - val_accuracy: 0.8465 - 560ms/epoch - 895us/step
Epoch 3/30
625/625 - 1s - loss: 0.2933 - accuracy: 0.8852 - val_loss: 0.3498 - val_accuracy: 0.8527 - 559ms/epoch - 894us/step
Epoch 4/30
625/625 - 1s - loss: 0.2435 - accuracy: 0.9044 - val_loss: 0.3439 - val_accuracy: 0.8550 - 562ms/epoch - 899us/step
Epoch 5/30
625/625 - 1s - loss: 0.2081 - accuracy: 0.9197 - val_loss: 0.3512 - val_accuracy: 0.8538 - 560ms/epoch - 895us/step
Epoch 6/30
625/625 - 1s - loss: 0.1788 - accuracy: 0.9320 - val_loss: 0.3681 - val_accuracy: 0.8512 - 571ms/epoch - 913us/step
Epoch 7/30
625/625 - 1s - loss: 0.1584 - accuracy: 0.9423 - val_loss: 0.3864 - val_accuracy: 0.8514 - 570ms/epoch - 912us/step
Epoch 8/30
625/625 - 1s - loss: 0.1393 - accuracy: 0.9496 - val_loss: 0.4179 - val_accuracy: 0.8444 - 587ms/epoch

In [52]:
sentence = ["Hope you have a great day!", "Thanks for your help, I really appreciate your advice."]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

1/1 [==============================] - 0s 14ms/step
[[3.295381e-02]
 [7.169237e-11]]
